<a href="https://colab.research.google.com/github/kul32/ML_datasets/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.28.0
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers
!pip install pyTelegramBotAPI requests

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel,AutoTokenizer, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset
def process_csv(file_path):
    df = pd.read_csv(file_path)
    qa_pairs = []
    for index, row in df.iterrows():
        question = row['Question']
        answer = row['Answer']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")
    return qa_pairs
def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True,
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset = load_dataset("/content/TrainSet.csv", tokenizer)
valid_dataset = load_dataset("/content/TestSet.csv", tokenizer)
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=15, training_loss=2.0265814463297525, metrics={'train_runtime': 267.9338, 'train_samples_per_second': 0.448, 'train_steps_per_second': 0.056, 'total_flos': 7838760960000.0, 'train_loss': 2.0265814463297525, 'epoch': 3.0})

In [ ]:
model.save_pretrained("fine_tuned_universe_gpt2")

In [ ]:
import telebot
import requests
import pandas as pd
from transformers import GPT2LMHeadModel,AutoTokenizer, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset
TELEGRAM_TOKEN ='6145842250:AAFGbcqThRDmiNUb_yBwMh2CHkDcXuUuq64'
model_path = '/content/fine_tuned_universe_gpt2'
bot = telebot.TeleBot(TELEGRAM_TOKEN)
fine_tuned_model = GPT2LMHeadModel.from_pretrained('/content/fine_tuned_universe_gpt2')
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
  bot.reply_to(message, "WELCOME! THE SUPER AI BOT.")
@bot.message_handler(func=lambda message: True)
def generate_response(message):
  response = ask_question(message.text, fine_tuned_model, tokenizer)
  bot.reply_to(message,response)
def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()
    answer = answer.split("\n")[0]
    return answer
bot.polling()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
